In [15]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [37]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local")

In [35]:
df18=pd.read_csv("ESTADIC//ESTADIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("ESTADIC//ESTADIC_educacao_2021.csv", sep=';')

In [26]:
df18 = df18[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
df21 = df21[['Cod UF', 'uf', 'genero', 'idade', 'cor_raca',
        'escolaridade']]
        

In [27]:
df18['ano']=2018
df21['ano']=2021

In [28]:
df = pd.concat([df18,df21])

In [29]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [30]:
## criando faixa etária
df1= df

In [31]:
df1

,Cod UF,uf,genero,idade,cor_raca,escolaridade,ano,faixa_etaria
0,11,Rondônia,Feminino,53,Branca,Especialização,2018,Entre 50-64
1,12,Acre,Masculino,52,Branca,Mestrado,2018,Entre 50-64
2,13,Amazonas,Masculino,73,Branca,Mestrado,2018,Acima de 65
3,14,Roraima,Feminino,40,Branca,Especialização,2018,Entre 30-49
4,15,Pará,Feminino,54,Pardo,Doutorado,2018,Entre 50-64
5,16,Amapá,Feminino,54,Pardo,Mestrado,2018,Entre 50-64
6,17,Tocantins,Feminino,44,Branca,Especialização,2018,Entre 30-49
7,21,Maranhão,Masculino,36,Branca,Mestrado,2018,Entre 30-49
8,22,Piauí,Masculino,38,Branca,Ensino superior completo,2018,Entre 30-49
9,23,Ceará,Masculino,39,Branca,Mestrado,2018,Entre 30-49


In [32]:
### Dataframe do mapa


df1 = df1.rename(columns={'Cod UF':'cod_uf'})
df1 = df1[['ano','cod_uf', 'uf', 'genero', 'faixa_etaria', 'cor_raca', 'escolaridade']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 26
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ano           54 non-null     int64   
 1   cod_uf        54 non-null     int64   
 2   uf            54 non-null     object  
 3   genero        54 non-null     object  
 4   faixa_etaria  54 non-null     category
 5   cor_raca      54 non-null     object  
 6   escolaridade  54 non-null     object  
dtypes: category(1), int64(2), object(4)
memory usage: 3.2+ KB


In [50]:
cod_uf = pd.read_csv('MUNIC//MUNIC_educacao_2021.csv', sep=';')[['UF','Cod UF']]

In [54]:
x= cod_uf.pivot_table(columns=('UF','Cod UF'), aggfunc='size')


In [59]:
cod_uf = pd.DataFrame(x).reset_index()[['UF','Cod UF']]

In [61]:
df1 = df1.merge(cod_uf, right_on='Cod UF',left_on='cod_uf')

In [62]:
df1['cor_raca'] = np.where(df1['cor_raca']=='Pardo','Parda',df1['cor_raca'])

In [64]:
df1 = df1.rename(columns={'UF':'sigla_uf'})

In [66]:
df1= df1[['ano', 'sigla_uf','cod_uf', 'uf', 'genero', 'faixa_etaria', 'cor_raca',
       'escolaridade']]

In [67]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ano           54 non-null     int64   
 1   sigla_uf      54 non-null     object  
 2   cod_uf        54 non-null     int64   
 3   uf            54 non-null     object  
 4   genero        54 non-null     object  
 5   faixa_etaria  54 non-null     category
 6   cor_raca      54 non-null     object  
 7   escolaridade  54 non-null     object  
dtypes: category(1), int64(2), object(5)
memory usage: 3.6+ KB


In [68]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [69]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
    bigquery.SchemaField('sigla_uf','STRING',description='sigla da UF'),
    bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
     bigquery.SchemaField('uf','STRING',description='Nome da UF'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')

 ]


In [70]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_educacao_mapa')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=555f50ed-03f5-4a34-b21d-3ae2de8cbd14>